# Pydantic Tutorial

## 🧩 Concept 1: What is Pydantic?

Pydantic is a Python library for data validation and settings management using Python type hints.
It allows you to define data models (like schemas) that ensure the input data matches the expected types — automatically converting and validating where possible.

---

## 🧱 Concept 2: Understanding BaseModel
What it is:

BaseModel is the core class in Pydantic from which all your data models inherit.
It gives your model:

* automatic data validation
* type coercion (conversion)
* serialization/deserialization
* and a rich set of helper methods like .dict(), .json(), .model_dump() (in v2), etc.

| Feature               | Description                                                     |
| --------------------- | --------------------------------------------------------------- |
| **Validation**        | Ensures incoming data matches field types                       |
| **Parsing**           | Automatically converts compatible types (e.g., `'123'` → `123`) |
| **Serialization**     | Converts models to dicts or JSON for storage or APIs            |
| **Immutability**      | Optional (`frozen=True`) to make models read-only               |
| **Model composition** | You can nest models within other models                         |

---

In [2]:
from pydantic import BaseModel

In [3]:
class Person(BaseModel):
    name: str
    age: int    

In [5]:
p1 = Person(name='akshay', age=20)

In [6]:
p2 = Person(name='someone', age='twenty')

ValidationError: 1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='twenty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

In [7]:
p1.json()

/var/folders/vq/y7pfb4bs4n78hbwj2701py6r0000gp/T/ipykernel_58676/2879298808.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  p1.json()


'{"name":"akshay","age":20}'

In [8]:
p1.model_dump_json()

'{"name":"akshay","age":20}'

In [9]:
p1.dict()

/var/folders/vq/y7pfb4bs4n78hbwj2701py6r0000gp/T/ipykernel_58676/3992612864.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  p1.dict()


{'name': 'akshay', 'age': 20}

In [12]:
p1.model_dump()

{'name': 'akshay', 'age': 20}

## ⚡ Concept 3: Type Coercion and Validation Rules

Pydantic doesn’t just check your data types — it also **tries to convert** compatible values into the right types automatically before raising an error.
This is called **type coercion**.

---

### 🧩 Example: Type Coercion

```python
from pydantic import BaseModel

class Product(BaseModel):
    id: int
    name: str
    price: float
    in_stock: bool

# Let's see what happens
p = Product(id="123", name="Widget", price="19.99", in_stock="true")
print(p)
print(p.model_dump())
```

✅ **Output:**

```
id=123 name='Widget' price=19.99 in_stock=True
{'id': 123, 'name': 'Widget', 'price': 19.99, 'in_stock': True}
```

👉 Notice that:

* `"123"` → `123` (converted to int)
* `"19.99"` → `19.99` (converted to float)
* `"true"` → `True` (converted to bool)

So, Pydantic will do its best to *coerce* data into the declared types whenever possible.

---

### ❌ When It Fails

If the value *cannot reasonably be converted*, Pydantic raises a `ValidationError`:

```python
Product(id="abc", name="Widget", price="19.99", in_stock="true")
```

🚨 Output:

```
pydantic_core._pydantic_core.ValidationError: 1 validation error for Product
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc']
```

---

### 🔒 Strict Mode (optional)

If you want **no coercion at all**, you can enforce *strict types*:

```python
from pydantic import BaseModel, StrictInt, StrictBool

class StrictProduct(BaseModel):
    id: StrictInt
    price: float
    in_stock: StrictBool

StrictProduct(id="123", price=9.99, in_stock="true")
```

This will **fail**, because `id` and `in_stock` aren’t *exact* types.

---

In [16]:
# type coercion example
from pydantic import BaseModel, StrictInt, StrictBool, ValidationError

class Car(BaseModel):
    brand: str
    model: str
    year: int
    is_electric: bool

class StrictCar(BaseModel):
    brand: str
    model: str
    year: StrictInt
    is_electric: StrictBool


In [17]:
simple_car = Car(
    brand = "honda",
    model = "cr-v",
    year = "2025",
    is_electric = "false"
)

In [18]:
try:
    strict_car = StrictCar(
        brand = "honda",
        model = "cr-v",
        year = "2025",
        is_electric = "false"        
    )
except ValidationError as e:
    print(e)

2 validation errors for StrictCar
year
  Input should be a valid integer [type=int_type, input_value='2025', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_type
is_electric
  Input should be a valid boolean [type=bool_type, input_value='false', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/bool_type


## 🧩 Concept 4: Default Values and Optional Fields

When you work with data from APIs or databases, sometimes certain fields are missing or have default values.
Pydantic makes handling this effortless.

| Field Type                | Behavior                                                       |
| ------------------------- | -------------------------------------------------------------- |
| `x: int`                  | Required field                                                 |
| `x: int = 10`             | Has a default value                                            |
| `x: Optional[int]`        | Can be `None`, but still required unless a default is provided |
| `x: Optional[int] = None` | Optional field that defaults to `None`                         |

💡 To indicate a field is optional, remember to include `from typing import Optional`!

In [20]:
from typing import Optional

class Product(BaseModel):
    name: str
    id: int
    description: Optional[str] = 'Coming soon..'
    on_sale: bool = False

In [22]:
proper_product = Product(
    name = 'T-Shirt',
    id = 123,
    description = 'Kids T-Shirt',
    on_sale = True
)
proper_product.model_dump()

{'name': 'T-Shirt', 'id': 123, 'description': 'Kids T-Shirt', 'on_sale': True}

In [23]:
product_with_defaults = Product(
    name = 'T-Shirt',
    id = 123
)
product_with_defaults.model_dump()

{'name': 'T-Shirt',
 'id': 123,
 'description': 'Coming soon..',
 'on_sale': False}

## 🧩 Concept 5: Nested Models (a.k.a. Compositional Data Models)

In real-world applications, data is often hierarchical.
For example, a user might have multiple addresses, or an order might contain multiple products.

Pydantic makes this clean and easy by allowing one model to be used as a field inside another.

In [24]:
from typing import List

class Author(BaseModel):
    name: str
    email: str

class Book(BaseModel):
    title: str
    authors: List[Author]

In [26]:
a1 = Author(
    name = 'Agatha Christie',
    email = 'achristie@gmail.com'
)
a2 = Author(
    name = 'Arthur Conan Doyle',
    email = 'acdoyle@gmail.com'
)

In [27]:
b1 = Book(
    title = 'The Strange Case of Missing Hound',
    authors = [a1, a2]
)

In [28]:
b1.model_dump()

{'title': 'The Strange Case of Missing Hound',
 'authors': [{'name': 'Agatha Christie', 'email': 'achristie@gmail.com'},
  {'name': 'Arthur Conan Doyle', 'email': 'acdoyle@gmail.com'}]}

Here is a slight de-tour. Before going to next concept, let us understand the need and importance of `typing` library.

---

## 🧩 Concept: The Role of the `typing` Module in Pydantic

Pydantic relies heavily on **type hints**, and the `typing` module provides the **building blocks** for expressing *complex data types* in Python.

---

### ⚙️ What `typing` Does

The `typing` module (built into Python 3.5+) allows you to **annotate variables, function parameters, and class attributes with type information**.

Pydantic reads these type hints to:

* know what type each field should be,
* automatically validate and convert input values, and
* give you smart autocompletion and static type checking (e.g., with VSCode or `mypy`).

---

### 🧠 Common Typing Constructs You’ll See

| Typing Type       | Example                      | Meaning                                        |
| ----------------- | ---------------------------- | ---------------------------------------------- |
| `List[int]`       | `scores: List[int]`          | A list of integers                             |
| `Dict[str, int]`  | `inventory: Dict[str, int]`  | A dict with string keys and int values         |
| `Optional[str]`   | `middle_name: Optional[str]` | A string or `None`                             |
| `Union[int, str]` | `id: Union[int, str]`        | Can be an int *or* a str                       |
| `Tuple[str, int]` | `entry: Tuple[str, int]`     | A tuple with exactly two elements: str and int |
| `Set[str]`        | `tags: Set[str]`             | A set of unique strings                        |
| `Any`             | `metadata: Any`              | Accepts *any* type (least strict)              |

---

### 💡 Why `typing` Matters in Pydantic

1. **Validation:**
   Pydantic checks incoming data types based on your annotations.
   Without `typing`, it wouldn’t know what structure to expect.

2. **Auto-conversion:**
   For example, `grades=["1", "2", "3"]` → `[1, 2, 3]`.

3. **Static Type Checking:**
   Tools like `mypy` can catch type mistakes *before runtime*.

4. **Clearer Code:**
   Your models serve as self-documenting schemas — easy to read, easy to maintain.

---

In [29]:
# example for `typing`
from typing import List, Optional, Dict

In [30]:
class Order(BaseModel):
    id: int
    items: List[str]
    notes: Optional[str] = None
    metadata: Optional[Dict[str, str]] = None

In [32]:
o1 = Order(
    id = 1,
    items = ['apple', 'banana']
)
o1.model_dump()

{'id': 1, 'items': ['apple', 'banana'], 'notes': None, 'metadata': None}

In [34]:
o2 = Order(
    id = 1,
    items = ['apple', 'banana'],
    notes = 'Get some raw bananas',
    metadata = {
        "requestedBy": "customer 1"
    }
)
o2.model_dump()

{'id': 1,
 'items': ['apple', 'banana'],
 'notes': 'Get some raw bananas',
 'metadata': {'requestedBy': 'customer 1'}}

## 🧩 Concept 6: Custom Validators 🧠

Even though Pydantic does automatic type checking, sometimes you need **custom rules**.
For example:

* A username shouldn’t contain spaces.
* A price must be positive.
* A date must be in the past.

That’s where **validators** come in.

---

### ⚙️ What is a Validator?

A validator is a **method inside your model** that runs automatically after type validation.
You decorate it with `@field_validator` (in Pydantic v2) or `@validator` (in v1).

It lets you define *custom logic* for validating or transforming a single field — or multiple fields together.

---

### 🧩 Multiple Fields Validation

Sometimes you need to validate a combination of fields (like “end_date must be after start_date”).
You can use a **model validator** for that:

```python
from datetime import date
from pydantic import BaseModel, model_validator

class Event(BaseModel):
    name: str
    start_date: date
    end_date: date

    @model_validator(mode="after")
    def check_dates(self):
        if self.end_date < self.start_date:
            raise ValueError("end_date must be after start_date")
        return self
```

---

## 🧩 Validator Modes in Pydantic v2

Both **`@field_validator`** and **`@model_validator`** decorators have a `mode` parameter that tells Pydantic **when** to run your validation function in the data-processing lifecycle.

---

### ⚙️ 1. `mode="before"`

This runs **before** type coercion and standard validation.
You’ll see the **raw input** exactly as provided by the user — before Pydantic converts it.

Use it when:

* You want to **preprocess** or **normalize** data before validation.
* You need to accept “messy” inputs (e.g., strings that should become ints or lists).

#### Example

```python
from pydantic import BaseModel, field_validator

class Product(BaseModel):
    tags: list[str]

    @field_validator("tags", mode="before")
    @classmethod
    def split_comma_string(cls, v):
        # If the user passes a string, turn it into a list
        if isinstance(v, str):
            return [x.strip() for x in v.split(",")]
        return v

p = Product(tags="electronics, gadgets, sale")
print(p)
```

✅ Output:

```
tags=['electronics', 'gadgets', 'sale']
```

---

### ⚙️ 2. `mode="after"`

This runs **after** Pydantic has already coerced and validated the data types.
You’ll get **typed and clean** data.

Use it when:

* You want to check constraints that depend on type-correct values (e.g., numbers, dates).
* You want to raise validation errors after Pydantic’s own checks.

#### Example

```python
from pydantic import BaseModel, field_validator

class Order(BaseModel):
    quantity: int

    @field_validator("quantity", mode="after")
    @classmethod
    def positive_quantity(cls, v):
        if v <= 0:
            raise ValueError("Quantity must be positive")
        return v
```

If `quantity` is `"10"` (string), Pydantic converts it to `10` first, then runs your validator.

---

### ⚙️ 3. (No Mode = Default)

If you don’t specify a mode, the default depends on **which validator** you’re using:

| Validator Type     | Default Mode | Typical Use                           |
| ------------------ | ------------ | ------------------------------------- |
| `@field_validator` | `"after"`    | Validate clean, type-correct values   |
| `@model_validator` | `"after"`    | Validate relationships between fields |


### ⚙️ Bonus: `mode="wrap"`

This one’s a bit more advanced and less commonly used — but powerful.
It lets you **wrap** the entire validation process for a field or model, giving you full control.

```python
@field_validator("price", mode="wrap")
def wrap_validation(cls, v, handler):
    print("Before validation:", v)
    result = handler(v)  # runs normal validation
    print("After validation:", result)
    return result
```

This is great for logging, tracing, or adding shared logic around validation.

---

### 🧠 Summary Table

| Mode       | Runs When                  | Data Passed In         | Typical Use                   |
| ---------- | -------------------------- | ---------------------- | ----------------------------- |
| `"before"` | Before Pydantic validation | Raw input              | Preprocessing, data cleanup   |
| `"after"`  | After validation           | Clean, typed values    | Logical/constraint validation |
| `"wrap"`   | Around validation          | Both raw and validated | Logging or full control flow  |

---

In [54]:
from pydantic import field_validator, ValidationError

# let's try all the validations in one class
class Address(BaseModel):
    number: int
    street: str
    city: str
    zipcode: int

    @field_validator('zipcode', mode='wrap')
    @classmethod
    def zipcode_check(cls, zipcode, handler):
        # make sure it is an int
        if isinstance(zipcode, int) or isinstance(zipcode, str):
            result = handler(zipcode)
            # check zipcode is less than 100000
            if result < 100000:
                return result
            else:
                raise ValueError('Zipcode has to be less than 100000')
        else:
            raise ValueError('Zipcode can be a string or an int')
    
class Person(BaseModel):
    name: str
    age: int
    hobbies: Optional[List[str]] = []
    addresses: List[Address]

    # let's make sure the person is 18 or up
    @field_validator('age', mode='after')
    @classmethod
    def is_adult(cls, age):
        if age < 18:
            raise ValueError("Age must be greater than 18")
        return age

    @field_validator('hobbies', mode='before')
    @classmethod
    def parse_hobbies(cls, hobbies):
        if isinstance(hobbies, str):            
            return [x.strip() for x in hobbies.split(',')]            
        return hobbies            

In [55]:
# this will thrown an error for age
try:
    person1 = Person(
        name = 'Akshay',
        age = 17
    )
except Exception as e:
    print(e)

2 validation errors for Person
age
  Value error, Age must be greater than 18 [type=value_error, input_value=17, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error
addresses
  Field required [type=missing, input_value={'name': 'Akshay', 'age': 17}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing


In [56]:
# this will parse hobbies properly
try:
    person1 = Person(
        name = 'Akshay',
        age = 21,
        hobbies = "chess, photography, mountain biking"
    )
except Exception as e:
    print(e)

1 validation error for Person
addresses
  Field required [type=missing, input_value={'name': 'Akshay', 'age':...raphy, mountain biking'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing


In [57]:
# now let's add address
try:
    person1 = Person(
        name = 'Akshay',
        age = 21,
        hobbies = "chess, photography, mountain biking",
        addresses = [
            Address(
                number = 10,
                street = 'Lincoln',
                city = 'Cupertino',
                zipcode = 101230123
            )
        ]
    )
except Exception as e:
    print(e)

1 validation error for Address
zipcode
  Value error, Zipcode has to be less than 100000 [type=value_error, input_value=101230123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


In [59]:
# finally, let's get a fully valid object
try:
    person1 = Person(
        name = 'Akshay',
        age = 21,
        hobbies = "chess, photography, mountain biking",
        addresses = [
            Address(
                number = 10,
                street = 'Lincoln',
                city = 'Cupertino',
                zipcode = 92010
            )
        ]
    )
except Exception as e:
    print(e)

person1.model_dump()

{'name': 'Akshay',
 'age': 21,
 'hobbies': ['chess', 'photography', 'mountain biking'],
 'addresses': [{'number': 10,
   'street': 'Lincoln',
   'city': 'Cupertino',
   'zipcode': 92010}]}

## 🧩 Concept 7: Model Configuration and Immutability

Every Pydantic model has a special attribute called `model_config`, where you define how it should behave globally.

### ⚙️ 1. Allowing or Forbidding Extra Fields

By default, Pydantic **forbids** unexpected fields that aren’t defined in your model.

```python
from pydantic import BaseModel, ConfigDict

class User(BaseModel):
    model_config = ConfigDict(extra="forbid")
    id: int
    name: str

User(id=1, name="Akshay", age=30)
```

🚨 Raises:

```
ValidationError: extra fields not permitted
```

But sometimes, you want to **ignore** or **store** those extra fields.

```python
class FlexibleUser(BaseModel):
    model_config = ConfigDict(extra="allow")
    id: int
    name: str

u = FlexibleUser(id=1, name="Akshay", age=30)
print(u.model_extra)
```

✅ Output:

```
{'age': 30}
```

---

### ⚙️ 2. Aliases (Renaming Fields)

You can define aliases to accept different input names for the same field — helpful for working with inconsistent APIs or databases. Note that this will enforce the input contains `productName` and not `product_name`.

```python
class Product(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    product_name: str
    price: float

# Accepts JSON-like data with alias
data = {"productName": "Laptop", "price": 1200.0}
p = Product(**data)
print(p)
```

✅ Output:

```
product_name='Laptop' price=1200.0
```

*(You can define explicit aliases too with `Field(alias="productName")`.)* Here's an example.

```python
from pydantic import BaseModel, Field

class Product(BaseModel):
    product_name: str = Field(alias="productName")
    price: float
```

If you want **both** `product_name` and `productName` to be supported, use:

```python
class Product(BaseModel):
    model_config = {"populate_by_name": True}
    product_name: str = Field(alias="productName")
```    

---

### ⚙️ 3. Immutability (Frozen Models)

You can make your model **read-only** (immutable) by setting `frozen=True`.

```python
class ConfiguredUser(BaseModel):
    model_config = ConfigDict(frozen=True)
    id: int
    name: str

user = ConfiguredUser(id=1, name="John")
user.name = "Jim"  # ❌ This will raise a TypeError
```

This is especially useful when you want your models to behave like **safe, hashable data objects** (e.g., for caching).

---

### 🧠 Common ConfigDict Options

| Option                 | Description                                       |
| ---------------------- | ------------------------------------------------- |
| `extra`                | `"ignore"`, `"forbid"`, or `"allow"` extra fields |
| `frozen`               | Makes model immutable                             |
| `populate_by_name`     | Allows using field aliases for initialization     |
| `validate_assignment`  | Re-validates data on attribute update             |
| `str_strip_whitespace` | Strips whitespace from strings                    |
| `str_to_lower`         | Converts strings to lowercase automatically       |

---

In [60]:
# let us update our author and adddress with the additional validations

from pydantic import field_validator, ValidationError, Field, BaseModel

# let's try all the validations in one class
class Address(BaseModel):
    number: int
    street: str
    city: str
    zipcode: int

    @field_validator('zipcode', mode='wrap')
    @classmethod
    def zipcode_check(cls, zipcode, handler):
        # make sure it is an int
        if isinstance(zipcode, int) or isinstance(zipcode, str):
            result = handler(zipcode)
            # check zipcode is less than 100000
            if result < 100000:
                return result
            else:
                raise ValueError('Zipcode has to be less than 100000')
        else:
            raise ValueError('Zipcode can be a string or an int')
    
class Person(BaseModel):
    model_config = {
        "frozen": True,
        "populate_by_name": True
    }
    first_name: str = Field(alias="firstName")
    last_name: str = Field(alias="lastName")
    age: int
    hobbies: Optional[List[str]] = []
    addresses: List[Address]

    # let's make sure the person is 18 or up
    @field_validator('age', mode='after')
    @classmethod
    def is_adult(cls, age):
        if age < 18:
            raise ValueError("Age must be greater than 18")
        return age

    @field_validator('hobbies', mode='before')
    @classmethod
    def parse_hobbies(cls, hobbies):
        if isinstance(hobbies, str):            
            return [x.strip() for x in hobbies.split(',')]            
        return hobbies            

In [66]:
# finally, let's get a fully valid object
try:
    person1 = {
        "firstName": "Sherlock",
        "lastName": "Holmes",
        "age": 35,
        "hobbies": "sleeping, acting, investigating",
        "addresses": [
            {
                "number": "221",
                "street": "Baker Street",
                "city": "London",
                "zipcode": "95212"
            }
        ]
    }
    p1 = Person(**person1)
except Exception as e:
    print(e)
p1.model_dump()    

{'first_name': 'Sherlock',
 'last_name': 'Holmes',
 'age': 35,
 'hobbies': ['sleeping', 'acting', 'investigating'],
 'addresses': [{'number': 221,
   'street': 'Baker Street',
   'city': 'London',
   'zipcode': 95212}]}

## 🧠 Python Dictionay Unpacking

Notice this line:

```python
p1 = Person(**person1)
```

This is a _pure python_ dictionary unpacking syntax. It says, take each `name=value` pair from the dictionary and pass it to the function/class instantiation.

So in this case, this is equivalent to saying the following:

```python
p1 = Person(
    firstName="Sherlock",
    lastName="Holmes",
    .. and so on ..
)
```

Using this concept, it is this possible to load a pydantic object directly from a JSON.

### ⚙️ Bonus: You can also use `.model.validate()`

Instead of using `p1 = Person(**person1)`, you can also use :

```python
p1 = Person.model_validate(person1)
```


In [67]:
# let's try the model_validate method
p1 = Person.model_validate(person1)

## 🧩 Concept 8: Model Serialization & JSON Conversion

After you validate and work with a `BaseModel`, you’ll almost always need to **export it** — for APIs, logging, or storage.
Pydantic makes this simple through a set of methods for converting models into dictionaries or JSON strings.

---

### ⚙️ 1. `.model_dump()`

This turns your model into a plain **Python dict**.

You can also use:

```python
p.model_dump(by_alias=True)
```

to switch back to the alias names:

```python
{'productName': 'Laptop', 'price': 999.99, 'in_stock': True}
```

---

### ⚙️ 2. `.model_dump_json()`

Exports directly to a **JSON string**.

```json
{"product_name":"Laptop","price":999.99,"in_stock":true}
```
Pass `by_alias=True` here as well if your API expects camelCase field names.

---

### ⚙️ 3. Filtering Fields

You can control what to include or exclude:

```python
p.model_dump(include={"product_name", "price"})
p.model_dump(exclude={"in_stock"})
```

You can even combine these with nested models:

```python
p.model_dump(exclude_unset=True)
```

👉 This excludes fields that still have their default values (useful for PATCH requests).

---


### ⚙️ 4. `copy()`

If you want to create a modified version without touching the original object:

```python
p2 = p.model_copy(update={"price": 899.99})
print(p2)
```

This respects immutability (`frozen=True`) and validates the updated data.

---

### 🧠 Summary Table

| Method                   | Returns            | Notes                                        |
| ------------------------ | ------------------ | -------------------------------------------- |
| `.model_dump()`          | `dict`             | Most common; friendly for logs or DB storage |
| `.model_dump_json()`     | `str` (JSON)       | Great for API responses                      |
| `.model_copy(update={})` | new model instance | Safe copy with modifications                 |
| `by_alias=True`          | —                  | Use alias names (e.g., camelCase)            |
| `exclude_unset=True`     | —                  | Skip default fields                          |

---

In [69]:
# let's try the serialization

class Book(BaseModel):
    book_title: str = Field(alias="bookTitle")
    pages: int
    price: float
    in_stock: bool = True

In [70]:
b = {
    "bookTitle": "Harry Potter",
    "pages": 300,
    "price": 123.25
}
book = Book.model_validate(b)

In [75]:
book.model_dump()

{'book_title': 'Harry Potter', 'pages': 300, 'price': 123.25, 'in_stock': True}

In [76]:
book.model_dump_json()

'{"book_title":"Harry Potter","pages":300,"price":123.25,"in_stock":true}'

In [77]:
book.model_dump_json(by_alias=True)

'{"bookTitle":"Harry Potter","pages":300,"price":123.25,"in_stock":true}'

In [78]:
book.model_dump_json(by_alias=True, exclude_unset=True)

'{"bookTitle":"Harry Potter","pages":300,"price":123.25}'

In [79]:
book.model_dump_json(by_alias=True, exclude={"price"})

'{"bookTitle":"Harry Potter","pages":300,"in_stock":true}'

In [81]:
# let's try a price update with model copy
new_book = book.model_copy(update={"price": 456.78})
new_book.model_dump()

{'book_title': 'Harry Potter', 'pages': 300, 'price': 456.78, 'in_stock': True}

## 🧩 Concept 9: Computed Fields and Property Transformations

Sometimes you want a field that isn’t stored in your input data but is *computed* from other fields.
For example, a user’s full name derived from first + last name, or a discounted price based on `price` and `discount`.

---

### ⚙️ 1. Using `@computed_field` ( Pydantic v2 )

```python
from pydantic import BaseModel, computed_field

class Product(BaseModel):
    name: str
    price: float
    discount: float = 0.0   # percent discount

    @computed_field
    @property
    def discounted_price(self) -> float:
        return self.price * (1 - self.discount / 100)
```

When you create an instance:

```python
p = Product(name="Laptop", price=1000, discount=10)
print(p.discounted_price)           # 900.0
print(p.model_dump())
```

✅ Output:

```python
{'name': 'Laptop', 'price': 1000.0, 'discount': 10.0, 'discounted_price': 900.0}
```

**Notes**

* The value is computed automatically.
* You don’t need to pass it as input.
* It appears in `.model_dump()` and `.model_dump_json()` by default.

---

### ⚙️ 2. Why `@computed_field` and not just a property?

Plain Python properties work fine for runtime access, but Pydantic:

* skips them in `.model_dump()` or JSON serialization by default.
* can’t treat them as model fields (no validation, aliasing, etc.).

Using `@computed_field` tells Pydantic:

> “Include this property in dumps and serialization as if it were a real field.”

---

### ⚙️ 3. Customizing Behavior

You can hide computed fields from serialization if needed:

```python
@computed_field(return_type=float, alias="finalPrice", repr=False)
@property
def discounted_price(self):
    return self.price * (1 - self.discount / 100)
```

* `return_type` lets you specify the type explicitly.
* `alias` lets you rename it for output (e.g., camelCase in APIs).
* `repr=False` hides it from `print(model)` but still includes it in `.model_dump()` if you want.

---

In [103]:
from pydantic import computed_field

# let's try a computed filed now
class Customer(BaseModel):
    model_config = {     
        "populate_by_name": True
    }
    first_name: str = Field(alias="firstName")
    last_name: str = Field(alias="lastName")

    @computed_field(return_type=str, alias="fullName", repr=True) # show this in the model_dump
    @property
    def get_full_name(self) -> str :
        return self.last_name + ", " + self.first_name
    

In [104]:
person = {
    "firstName": "James",
    "lastName": "Bond"
}
p1 = Customer.model_validate(person)

In [105]:
p1.model_dump()

{'first_name': 'James', 'last_name': 'Bond', 'get_full_name': 'Bond, James'}

In [108]:
# note the field is named `get_full_name` which is ugly. so use the alias
import json

p1.model_dump_json(by_alias=True)

'{"firstName":"James","lastName":"Bond","fullName":"Bond, James"}'